In [20]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import cplex
import seaborn as sns

In [21]:
# buy and hold strategy

def strat_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    x_optimal = x_init
    cash_optimal = cash_init
    
    return x_optimal, cash_optimal

In [22]:
# equally weighted strategy with buy and hold

def strat_equally_weighted_with_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    V_init = np.dot(cur_prices, x_init) + cash_init
    x_optimal = np.array([505., 6636.,  539.,   46., 2566.,  994., 1540.,  976., 1287.,
        476., 1937., 1240., 2081., 1108., 2620., 1020.,  368.,   31.,
       1302., 1108.])
    
    cash_optimal = V_init - np.dot(cur_prices, x_optimal)
    
    return x_optimal, cash_optimal

In [23]:
# minimum variance strategy with buy and hold

def strat_min_variance_with_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    V_init = np.dot(cur_prices, x_init) + cash_init
    x_optimal = np.array([0., 23871.,     0.,    79.,     0.,     0.,     0.,  9868.,
           0.,     0.,  2050.,     0.,     0.,     0.,     0.,     0.,
         150.,     0.,     0.,  3531.])
    
    cash_optimal = V_init - np.dot(cur_prices, x_optimal)
    
    return x_optimal, cash_optimal

In [24]:
# maximum Sharpe ratio strategy with buy and hold

def strat_max_Sharpe_with_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    V_init = np.dot(cur_prices, x_init) + cash_init
    x_optimal = np.array([0.,     0.,     0.,     0.,     0.,     0.,     0., 19433.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])
    
    cash_optimal = V_init - np.dot(cur_prices, x_optimal)

    return x_optimal, cash_optimal

In [25]:
# Input file
input_file_prices = 'Daily_closing_prices.csv'

# Read data into a dataframe
df = pd.read_csv(input_file_prices)

# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    temp = [int(x) for x in datestr.split('/')]
    return [temp[-1], temp[0], temp[1]]

dates_array = np.array(list(df['Date'].apply(convert_date_to_array)))
data_prices = df.iloc[:, 1:].to_numpy()
dates = np.array(df['Date'])
# Find the number of trading days in Nov-Dec 2018 and
# compute expected return and covariance matrix for period 1
day_ind_start0 = 0
day_ind_end0 = len(np.where(dates_array[:,0]==2018)[0])
cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
mu = np.mean(cur_returns0, axis = 0)
Q = np.cov(cur_returns0.T)

# Remove datapoints for year 2018
data_prices = data_prices[day_ind_end0:,:]
dates_array = dates_array[day_ind_end0:,:]
dates = dates[day_ind_end0:]

# Initial positions in the portfolio
init_positions = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 980, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20000])

# Initial value of the portfolio
init_value = np.dot(data_prices[0,:], init_positions)
print('\nInitial portfolio value = $ {}\n'.format(round(init_value, 2)))

# Initial portfolio weights
w_init = (data_prices[0,:] * init_positions) / init_value

# Number of periods, assets, trading days
N_periods = 6*len(np.unique(dates_array[:,0])) # 6 periods per year
N = len(df.columns)-1
N_days = len(dates)

# Annual risk-free rate for years 2019-2020 is 2.5%
r_rf = 0.025

# Number of strategies
strategy_functions = ['strat_buy_and_hold', 'strat_equally_weighted_with_buy_and_hold', 'strat_min_variance_with_buy_and_hold', 'strat_max_Sharpe_with_buy_and_hold']
strategy_names     = ['Buy and Hold', 'Equally Weighted Portfolio with buy and hold', 'Mininum Variance Portfolio with buy and hold', 'Maximum Sharpe Ratio Portfolio with buy and hold']
#N_strat = 1  # comment this in your code
N_strat = len(strategy_functions)  # uncomment this in your code
fh_array = [strat_buy_and_hold, strat_equally_weighted_with_buy_and_hold, strat_min_variance_with_buy_and_hold, strat_max_Sharpe_with_buy_and_hold]

portf_value = [0] * N_strat
x = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
cash = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
for period in range(1, N_periods+1):
   # Compute current year and month, first and last day of the period
   if dates_array[0, 0] == 19:
       cur_year  = 19 + math.floor(period/7)
   else:
       cur_year  = 2019 + math.floor(period/7)

   cur_month = 2*((period-1)%6) + 1
   day_ind_start = min([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val])
   day_ind_end = max([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val])
   print('\nPeriod {0}: start date {1}, end date {2}'.format(period, dates[day_ind_start], dates[day_ind_end]))
   
   # Prices for the current day
   cur_prices = data_prices[day_ind_start,:]

   # Execute portfolio selection strategies
   for strategy  in range(N_strat):

      # Get current portfolio positions
      if period == 1:
         curr_positions = init_positions
         curr_cash = 0
         portf_value[strategy] = np.zeros((N_days, 1))
      else:
         curr_positions = x[strategy, period-2]
         curr_cash = cash[strategy, period-2]

      # Compute strategy
      x[strategy, period-1], cash[strategy, period-1] = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

      # Verify that strategy is feasible (you have enough budget to re-balance portfolio)
      # Check that cash account is >= 0
      # Check that we can buy new portfolio subject to transaction costs

      while cash[strategy, period-1] < 0:
            
            V_init = np.dot(cur_prices, curr_positions) + curr_cash
            x_normalized = x[strategy][period-1]/np.sum(x[strategy][period-1])
            cash_extra = abs(cash[strategy][period-1])*x_normalized
            x_extra = np.ceil(cash_extra/cur_prices)
            x[strategy][period-1] = x[strategy][period-1] - x_extra
            txn_cost = 0.005*np.dot(cur_prices, abs(x[strategy][period-1]-curr_positions))
            cash[strategy][period-1] = V_init - np.sum(cur_prices*x[strategy][period-1]) - txn_cost

                
      # Compute portfolio value
      p_values = np.dot(data_prices[day_ind_start:day_ind_end+1,:], x[strategy, period-1]) + cash[strategy, period-1]
      portf_value[strategy][day_ind_start:day_ind_end+1] = np.reshape(p_values, (p_values.size,1))
      print('  Strategy "{0}", value begin = $ {1:.2f}, value end = $ {2:.2f}'.format( strategy_names[strategy], 
             portf_value[strategy][day_ind_start][0], portf_value[strategy][day_ind_end][0]))

      
   # Compute expected returns and covariances for the next period
   cur_returns = data_prices[day_ind_start+1:day_ind_end+1,:] / data_prices[day_ind_start:day_ind_end,:] - 1
   mu = np.mean(cur_returns, axis = 0)
   Q = np.cov(cur_returns.T)
    



Initial portfolio value = $ 1000070.06


Period 1: start date 01/02/2019, end date 02/28/2019
  Strategy "Buy and Hold", value begin = $ 1000070.06, value end = $ 1121179.83
  Strategy "Equally Weighted Portfolio with buy and hold", value begin = $ 1000070.06, value end = $ 1105977.48
  Strategy "Mininum Variance Portfolio with buy and hold", value begin = $ 1000070.06, value end = $ 1065808.21
  Strategy "Maximum Sharpe Ratio Portfolio with buy and hold", value begin = $ 1000070.06, value end = $ 1026475.08

Period 2: start date 03/01/2019, end date 04/30/2019
  Strategy "Buy and Hold", value begin = $ 1126131.27, value end = $ 1075001.89
  Strategy "Equally Weighted Portfolio with buy and hold", value begin = $ 1112519.43, value end = $ 1198618.65
  Strategy "Mininum Variance Portfolio with buy and hold", value begin = $ 1068773.79, value end = $ 1116497.29
  Strategy "Maximum Sharpe Ratio Portfolio with buy and hold", value begin = $ 1027189.40, value end = $ 1041818.94

Period 3: 